In [6]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [8]:
# Read eligibility matrix
eligibility_df = pd.read_csv('men.csv', index_col=0)

# Read adjacency matrix
adjacency_df = pd.read_csv('exclusions.csv', index_col=0)

In [10]:
# helper functions
def is_eligible(person, task) -> bool:
    return eligibility_df.at[person, task] == 1.0

def rotate_list(lst, n):
    return lst[n:] + lst[:n]

In [24]:
# round-robin seed
seed = 1

# List of people and tasks
people = list(eligibility_df.index)
people = rotate_list(people, len(people) % seed)
tasks = list(eligibility_df.columns)

In [26]:
# Create the LP problem
prob = LpProblem("Task_Assignment", LpMaximize)

# Define decision variables
x = LpVariable.dicts("assign", ((person, task) for person in people for task in tasks), cat='Binary')
y = LpVariable.dicts("assigned", people, cat='Binary')

###
# Objective function: Maximize the number of unique people assigned
###
prob += lpSum(y[person] for person in people)

In [28]:
# Only assign eligible people
for person in people:
    for task in tasks:
        if not is_eligible(person, task):
            prob += x[(person, task)] == 0

# Ensure y is 1 if person is assigned to at least one task
for person in people:
    prob += y[person] <= lpSum(x[(person, task)] for task in tasks)
    # prob += y[person] >= 0.1 * lpSum(x[(person, task)] for task in tasks)

# Do not assign a person to two excluded tasks
for person in people:
    for task1 in tasks:
            for task2 in tasks:
                if task1 != task2 and adjacency_df.loc[task1, task2] == 1:
                    if is_eligible(person, task1) and is_eligible(person, task1):
                        prob += x[(person, task1)] + x[(person, task2)] <= 1

# Task limit constraints
for person in people:
    prob += lpSum(x[(person, task)] for task in tasks) <= 2

# Task assignment constraints: each task is assigned to exactly one person
for task in tasks:
    prob += lpSum(x[(person, task)] for person in people) == 1

In [30]:
# Solve the problem
result = prob.solve()

# Output the results
assignment = {}
for person in people:
    assigned_tasks = [task for task in tasks if x[(person, task)].varValue == 1]
    if assigned_tasks:
        assignment[person] = assigned_tasks

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/6dd49b3f9f844d40bfc588af096ae389-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/4z/l87fr4g16qb0rxtm42zcxtym0000gn/T/6dd49b3f9f844d40bfc588af096ae389-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3310 COLUMNS
At line 14161 RHS
At line 17467 BOUNDS
At line 18476 ENDATA
Problem MODEL has 3305 rows, 1008 columns and 8792 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 23 - 0.01 seconds
Cgl0002I 549 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 624 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 666 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 515 strengthened rows, 0 substitu

In [22]:
from collections import OrderedDict
schedule = OrderedDict({task : [person for person in assignment.keys() if task in assignment[person]][0] for task in tasks})
for key, value in list(schedule.items()):
    print(f"{key}: {value}")

first_opening_prayer: Tipton, Dayle
first_lesson: Smiley, Justin
first_song_leader: Vinson, Joe
announcements: Nunn, Jeff
song_leader: Tipton, Sam
scripture_reading: Stroik, Darrin
opening_prayer: Taylor, Keith
closing_prayer: Purcell, Tripp
table_lead_bread: Warren, David
table_lead_cup: Purcell, Lance
table_aid_bread: McAnear, Jaxan
table_aid_cup: Ledbetter, Patrick
lesson: Byers, Austin
wednesday_announcements: Nunn, Sam
wednesday_song_leader: McAlister, Benson
wednesday_opening_prayer: McAnear, Justus
wednesday_lesson: Scott, Rusty
wednesday_closing_prayer: Love, Chris
lords_supper_prep: Hight, Ben
usher: McAnear, Walker
alt_usher: Yontz, Trevor
security: Evans, Barret
sound_board_operator: McAlister, Grady
